In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, round

In [2]:
spark = SparkSession.builder.master("local[*]").appName("Basics 1").getOrCreate()

In [3]:
from pyspark.sql.functions import col, when, count, lag, avg, stddev
from pyspark.sql import functions as F

In [4]:
data = [("Dinkar", 24), ("Mandeep", 23), ("Raj", 22)]
columns = ["name", "age"]

In [5]:
data1 = [{"Name":"Dinkar", "Disg":"Data Scientist"},{"Name":"Nissarg", "Disg":"Teaching assistent"},{"Name":"Mandeep", "Disg":"Data Scientist"}]

In [6]:
df1 = spark.read.csv("./sales.csv", header=True, inferSchema=True, )

In [7]:
df1.select("gender", "age").show()

+------+---+
|gender|age|
+------+---+
|     M| 38|
|  NULL| 19|
|     F| 39|
|     F| 64|
|     M| 26|
|     F| 37|
|     F| 23|
|     F| 51|
|     F| 43|
|     M| 24|
|     F| 52|
|     F| 24|
|     F| 57|
|     M| 43|
|     F| 56|
|     M| 46|
|     F| 59|
|     F| 45|
|     M| 58|
|     F| 41|
+------+---+
only showing top 20 rows


In [8]:
nulls = df1.select(count(when(col("city").isNull(), "city"))).show()

+---------------------------------------------+
|count(CASE WHEN (city IS NULL) THEN city END)|
+---------------------------------------------+
|                                            4|
+---------------------------------------------+



In [9]:
# df1.join(df, on="id", how="inner")

In [10]:
df1.na.drop().show()

+--------+----------+-----------+------+---+---------+----------+--------------+--------+-----+--------+--------------+
|order_id|order_date|customer_id|gender|age|     city|   product|      category|quantity|price|discount|payment_method|
+--------+----------+-----------+------+---+---------+----------+--------------+--------+-----+--------+--------------+
|    1001|2024-03-29|       C072|     M| 38|  Kolkata|       Fan|Home Appliance|       3| 1160|       7|    Debit Card|
|    1003|2025-07-02|       C161|     F| 39|Ahmedabad|     Jeans|       Apparel|       4| 5611|      12|   Credit Card|
|    1005|2025-02-01|       C135|     M| 26|  Kolkata|       Fan|Home Appliance|       1|31851|       7|    Debit Card|
|    1006|2023-11-29|       C053|     F| 37|  Kolkata|     Shoes|      Footwear|       2|74269|       0|          Cash|
|    1007|2025-04-30|       C002|     F| 23|   Jaipur|Smartwatch|   Electronics|       4|36220|       5|    Debit Card|
|    1008|2024-01-17|       C218|     F|

In [11]:
nulls = df1.select([count(when(col(c).isNull(), c)) for c in df1.columns]).toPandas()
nulls

,count(CASE WHEN (order_id IS NULL) THEN order_id END),count(CASE WHEN (order_date IS NULL) THEN order_date END),count(CASE WHEN (customer_id IS NULL) THEN customer_id END),count(CASE WHEN (gender IS NULL) THEN gender END),count(CASE WHEN (age IS NULL) THEN age END),count(CASE WHEN (city IS NULL) THEN city END),count(CASE WHEN (product IS NULL) THEN product END),count(CASE WHEN (category IS NULL) THEN category END),count(CASE WHEN (quantity IS NULL) THEN quantity END),count(CASE WHEN (price IS NULL) THEN price END),count(CASE WHEN (discount IS NULL) THEN discount END),count(CASE WHEN (payment_method IS NULL) THEN payment_method END)
0,0,0,0,1,0,4,0,0,3,0,3,0


In [12]:
# df = spark.createDataFrame(data, columns)
# df.limit(20).collect()

In [13]:
# df = spark.createDataFrame(data1)
# df.limit(20).collect()

In [14]:
a = [1,2,3]
b = ["one","two","three"]

c = zip(a,b)

In [15]:
df1 = df1.withColumn(
    "discount_amount",
    round(col("price") * (col("discount") / 100), 2)
)

In [16]:
df1.show()

+--------+----------+-----------+------+---+---------+----------+--------------+--------+-----+--------+--------------+---------------+
|order_id|order_date|customer_id|gender|age|     city|   product|      category|quantity|price|discount|payment_method|discount_amount|
+--------+----------+-----------+------+---+---------+----------+--------------+--------+-----+--------+--------------+---------------+
|    1001|2024-03-29|       C072|     M| 38|  Kolkata|       Fan|Home Appliance|       3| 1160|       7|    Debit Card|           81.2|
|    1002|2025-01-12|       C150|  NULL| 19|Hyderabad|      Iron|Home Appliance|       4|44431|       7|          Cash|        3110.17|
|    1003|2025-07-02|       C161|     F| 39|Ahmedabad|     Jeans|       Apparel|       4| 5611|      12|   Credit Card|         673.32|
|    1004|2025-02-04|       C271|     F| 64|Bengaluru|      Iron|Home Appliance|    NULL|18731|      10|          Cash|         1873.1|
|    1005|2025-02-01|       C135|     M| 26|  Ko